# * VINSIGHT : Churn Subs
    Prepaid Churn Subs : TMH ยังใช้ไม่ได้ รอ verify ตัวเลข

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Import Transaction

In [2]:
''' Execute transaction '''


# Input parameter
v_start_date = 20240101
print(f'\nParameter input...')
print(f'   -> v_start_date: {v_start_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B1S000200' --Prepaid Churn Subs
        , 'TB1S000200' --Prepaid Churn Subs : TMH
        , 'DB1S000200' --Prepaid Churn Subs : DTAC
        
        , 'B2S010200' --Postpaid Churn Subs B2C
        , 'TB2S010200' --Postpaid Churn Subs B2C : TMH
        , 'DB2S010200' --Postpaid Churn Subs B2C : DTAC
        , 'B2S010201' --Postpaid Churn Subs Voluntary B2C
        , 'TB2S010201' --Postpaid Churn Subs Voluntary B2C : TMH
        , 'DB2S010201' --Postpaid Churn Subs Voluntary B2C : DTAC
        , 'B2S010202' --Postpaid Churn Subs Involuntary B2C
        , 'TB2S010202' --Postpaid Churn Subs Involuntary B2C : TMH
        , 'DB2S010202' --Postpaid Churn Subs Involuntary B2C : DTAC
        , 'TB2S010210' --Postpaid Churn Subs B2C : TMH (Most Used Location)
        , 'TB2S010211' --Postpaid Churn Subs Voluntary B2C : TMH (Most Used Location)
        , 'TB2S010212' --Postpaid Churn Subs Involuntary B2C : TMH (Most Used Location)
        
        , 'B2S020200' --Postpaid Churn Subs B2B
        , 'TB2S020200' --Postpaid Churn Subs B2B : TMH
        , 'DB2S020200' --Postpaid Churn Subs B2B : DTAC
        , 'B2S020201' --Postpaid Churn Subs Voluntary B2B
        , 'TB2S020201' --Postpaid Churn Subs Voluntary B2B : TMH
        , 'DB2S020201' --Postpaid Churn Subs Voluntary B2B : DTAC
        , 'B2S020202' --Postpaid Churn Subs Involuntary B2B
        , 'TB2S020202' --Postpaid Churn Subs Involuntary B2B : TMH
        , 'DB2S020202' --Postpaid Churn Subs Involuntary B2B : DTAC
        , 'TB2S020210' --Postpaid Churn Subs B2B : TMH (Most Used Location)
        , 'TB2S020211' --Postpaid Churn Subs Voluntary B2B : TMH (Most Used Location)
        , 'TB2S020212' --Postpaid Churn Subs Involuntary B2B : TMH (Most Used Location)
        
        , 'TSER13100' --FTTx Churn Subs
        , 'TSER13102' --FTTx Churn Subs Voluntary
        , 'TSER13103' --FTTx Churn Subs Involuntary
        , 'TSER13104' --FTTx Churn Subs Involuntary (Collection Forecast)
        , 'TSER14100' --TVS Churn Subs
        , 'TSER14102' --TVS Churn Subs Voluntary
        , 'TSER14103' --TVS Churn Subs Involuntary
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    print(f'\nDataFrame: {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...
   -> v_start_date: 20240101

Data as of 2025-06-18, 10:14:10

DataFrame: 18144 rows, 16 columns


In [3]:
''' Automate Currently Period '''

curr_yr = chk_src_df['TM_KEY_YR'].max()
prev_yr = chk_src_df['TM_KEY_YR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_qtr = chk_src_df['TM_KEY_QTR'].max()
prev_qtr = chk_src_df['TM_KEY_QTR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = chk_src_df['TM_KEY_MTH'].drop_duplicates().sort_values().shift().max().astype(int)

curr_wk = chk_src_df['TM_KEY_WK'].max()
prev_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift().max().astype(int)
last_3_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift(3).max().astype(int)

# curr_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].max().astype(int)
# prev_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].drop_duplicates().shift().max().astype(int)

## Overview by Period

In [4]:
''' Churn Subs Yearly '''

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'TB1S000200' #Prepaid Churn Subs : TMH
    , 'DB1S000200' #Prepaid Churn Subs : DTAC
    , 'B2S010200' #Postpaid Churn Subs B2C
    , 'TB2S010200' #Postpaid Churn Subs B2C : TMH
    , 'DB2S010200' #Postpaid Churn Subs B2C : DTAC
    , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    , 'TB2S010201' #Postpaid Churn Subs Voluntary B2C : TMH
    , 'DB2S010201' #Postpaid Churn Subs Voluntary B2C : DTAC
    , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TB2S010202' #Postpaid Churn Subs Involuntary B2C : TMH
    , 'DB2S010202' #Postpaid Churn Subs Involuntary B2C : DTAC
    # , 'TB2S010210' #Postpaid Churn Subs B2C : TMH (Most Used Location)
    # , 'TB2S010211' #Postpaid Churn Subs Voluntary B2C : TMH (Most Used Location)
    # , 'TB2S010212' #Postpaid Churn Subs Involuntary B2C : TMH (Most Used Location)
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    , 'TSER14102' #TVS Churn Subs Voluntary
    , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_sub_yearly_df = chk_src_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
churn_sub_yearly_df = churn_sub_yearly_df.loc[churn_sub_yearly_df['METRIC_CD'].isin(v_metric_list)]

churn_sub_yearly_df['PRE'] = np.where(churn_sub_yearly_df['METRIC_CD']=='B1S000200', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['PRE_T'] = np.where(churn_sub_yearly_df['METRIC_CD']=='TB1S000200', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['PRE_D'] = np.where(churn_sub_yearly_df['METRIC_CD']=='DB1S000200', churn_sub_yearly_df['P'], 0)

churn_sub_yearly_df['POST_B2C'] = np.where(churn_sub_yearly_df['METRIC_CD']=='B2S010200', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['POST_B2C_VL'] = np.where(churn_sub_yearly_df['METRIC_CD']=='B2S010201', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['POST_B2C_IVL'] = np.where(churn_sub_yearly_df['METRIC_CD']=='B2S010202', churn_sub_yearly_df['P'], 0)

churn_sub_yearly_df['POST_B2C_T'] = np.where(churn_sub_yearly_df['METRIC_CD']=='TB2S010200', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['POST_B2C_VL_T'] = np.where(churn_sub_yearly_df['METRIC_CD']=='TB2S010201', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['POST_B2C_IVL_T'] = np.where(churn_sub_yearly_df['METRIC_CD']=='TB2S010202', churn_sub_yearly_df['P'], 0)

churn_sub_yearly_df['POST_B2C_D'] = np.where(churn_sub_yearly_df['METRIC_CD']=='DB2S010200', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['POST_B2C_VL_D'] = np.where(churn_sub_yearly_df['METRIC_CD']=='DB2S010201', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['POST_B2C_IVL_D'] = np.where(churn_sub_yearly_df['METRIC_CD']=='DB2S010202', churn_sub_yearly_df['P'], 0)

churn_sub_yearly_df['FTTX'] = np.where(churn_sub_yearly_df['METRIC_CD']=='TSER13100', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['FTTX_VL'] = np.where(churn_sub_yearly_df['METRIC_CD']=='TSER13102', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['FTTX_IVL'] = np.where(churn_sub_yearly_df['METRIC_CD']=='TSER13103', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['FTTX_IVL(CF)'] = np.where(churn_sub_yearly_df['METRIC_CD']=='TSER13104', churn_sub_yearly_df['P'], 0)

churn_sub_yearly_df['TVS'] = np.where(churn_sub_yearly_df['METRIC_CD']=='TSER14100', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['TVS_VL'] = np.where(churn_sub_yearly_df['METRIC_CD']=='TSER14102', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['TVS_IVL'] = np.where(churn_sub_yearly_df['METRIC_CD']=='TSER14103', churn_sub_yearly_df['P'], 0)


# ''' Full '''
# churn_sub_yearly_df = churn_sub_yearly_df.groupby('TM_KEY_YR')\
#     .agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum'
#           , 'POST_B2C':'sum', 'POST_B2C_VL':'sum', 'POST_B2C_IVL':'sum'
#           , 'POST_B2C_T':'sum', 'POST_B2C_VL_T':'sum', 'POST_B2C_IVL_T':'sum'
#           , 'POST_B2C_D':'sum', 'POST_B2C_VL_D':'sum', 'POST_B2C_IVL_D':'sum'
#           , 'FTTX':'sum', 'FTTX_VL':'sum', 'FTTX_IVL':'sum'
#           , 'TVS':'sum', 'TVS_VL':'sum', 'TVS_IVL':'sum'})
# churn_sub_yearly_df = churn_sub_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
# churn_sub_yearly_df = churn_sub_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF'
#                                          , 'PRE', 'PRE_T', 'PRE_D'
#                                          , 'POST_B2C', 'POST_B2C_VL', 'POST_B2C_IVL'
#                                          , 'POST_B2C_T', 'POST_B2C_VL_T', 'POST_B2C_IVL_T'
#                                          , 'POST_B2C_D', 'POST_B2C_VL_D', 'POST_B2C_IVL_D'
#                                          , 'FTTX', 'FTTX_VL', 'FTTX_IVL'
#                                          , 'TVS', 'TVS_VL', 'TVS_IVL']]
# churn_sub_yearly_df_display = churn_sub_yearly_df.copy()
# churn_sub_yearly_df_display['ACTUAL_AS_OF'] = churn_sub_yearly_df_display['ACTUAL_AS_OF'].astype(int)
# mod_col_list = churn_sub_yearly_df_display.iloc[:, 3:].columns.tolist()
# for col in mod_col_list:
#     churn_sub_yearly_df_display[col] = churn_sub_yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
# print(f'\n{churn_sub_yearly_df_display.to_string(max_cols=100)}') #max_rows=1000


''' Overview '''
churn_sub_yearly_df = churn_sub_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'POST_B2C':'sum', 'POST_B2C_VL':'sum', 'POST_B2C_IVL':'sum', 'FTTX':'sum', 'FTTX_VL':'sum', 'FTTX_IVL':'sum', 'TVS':'sum', 'TVS_VL':'sum', 'TVS_IVL':'sum'})
churn_sub_yearly_df = churn_sub_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
churn_sub_yearly_df = churn_sub_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'POST_B2C', 'POST_B2C_VL', 'POST_B2C_IVL', 'FTTX', 'FTTX_VL', 'FTTX_IVL', 'TVS', 'TVS_VL', 'TVS_IVL']]
churn_sub_yearly_df_display = churn_sub_yearly_df.copy()
churn_sub_yearly_df_display['ACTUAL_AS_OF'] = churn_sub_yearly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = churn_sub_yearly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    churn_sub_yearly_df_display[col] = churn_sub_yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
churn_sub_yearly_df_display

,TM_KEY_YR,PPN_TM,ACTUAL_AS_OF,PRE,POST_B2C,POST_B2C_VL,POST_B2C_IVL,FTTX,FTTX_VL,FTTX_IVL,TVS,TVS_VL,TVS_IVL
0,2024,2025-06-18 01:06:16,20241231,"34,112,179","2,857,216","1,066,524","1,862,336","456,668","170,154","281,222","194,250","84,176","18,800"
1,2025,2025-06-18 01:06:16,20250616,"29,834,194","1,033,026","426,920","612,233","189,607","72,952","125,678","101,108","10,659",76


In [5]:
''' Churn Subs Quarterly '''

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'TB1S000200' #Prepaid Churn Subs : TMH
    , 'DB1S000200' #Prepaid Churn Subs : DTAC
    , 'B2S010200' #Postpaid Churn Subs B2C
    , 'TB2S010200' #Postpaid Churn Subs B2C : TMH
    , 'DB2S010200' #Postpaid Churn Subs B2C : DTAC
    , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    , 'TB2S010201' #Postpaid Churn Subs Voluntary B2C : TMH
    , 'DB2S010201' #Postpaid Churn Subs Voluntary B2C : DTAC
    , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TB2S010202' #Postpaid Churn Subs Involuntary B2C : TMH
    , 'DB2S010202' #Postpaid Churn Subs Involuntary B2C : DTAC
    # , 'TB2S010210' #Postpaid Churn Subs B2C : TMH (Most Used Location)
    # , 'TB2S010211' #Postpaid Churn Subs Voluntary B2C : TMH (Most Used Location)
    # , 'TB2S010212' #Postpaid Churn Subs Involuntary B2C : TMH (Most Used Location)
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    , 'TSER14102' #TVS Churn Subs Voluntary
    , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_sub_quarterly_df = chk_src_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
churn_sub_quarterly_df = churn_sub_quarterly_df.loc[churn_sub_quarterly_df['METRIC_CD'].isin(v_metric_list)]

churn_sub_quarterly_df['PRE'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='B1S000200', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['PRE_T'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='TB1S000200', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['PRE_D'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='DB1S000200', churn_sub_quarterly_df['P'], 0)

churn_sub_quarterly_df['POST_B2C'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='B2S010200', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['POST_B2C_VL'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='B2S010201', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['POST_B2C_IVL'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='B2S010202', churn_sub_quarterly_df['P'], 0)

churn_sub_quarterly_df['POST_B2C_T'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='TB2S010200', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['POST_B2C_VL_T'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='TB2S010201', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['POST_B2C_IVL_T'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='TB2S010202', churn_sub_quarterly_df['P'], 0)

churn_sub_quarterly_df['POST_B2C_D'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='DB2S010200', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['POST_B2C_VL_D'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='DB2S010201', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['POST_B2C_IVL_D'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='DB2S010202', churn_sub_quarterly_df['P'], 0)

churn_sub_quarterly_df['FTTX'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='TSER13100', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['FTTX_VL'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='TSER13102', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['FTTX_IVL'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='TSER13103', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['FTTX_IVL(CF)'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='TSER13104', churn_sub_quarterly_df['P'], 0)

churn_sub_quarterly_df['TVS'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='TSER14100', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['TVS_VL'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='TSER14102', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['TVS_IVL'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='TSER14103', churn_sub_quarterly_df['P'], 0)


# ''' Full '''
# churn_sub_quarterly_df = churn_sub_quarterly_df.groupby('TM_KEY_QTR')\
#     .agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum'
#           , 'POST_B2C':'sum', 'POST_B2C_VL':'sum', 'POST_B2C_IVL':'sum'
#           , 'POST_B2C_T':'sum', 'POST_B2C_VL_T':'sum', 'POST_B2C_IVL_T':'sum'
#           , 'POST_B2C_D':'sum', 'POST_B2C_VL_D':'sum', 'POST_B2C_IVL_D':'sum'
#           , 'FTTX':'sum', 'FTTX_VL':'sum', 'FTTX_IVL':'sum'
#           , 'TVS':'sum', 'TVS_VL':'sum', 'TVS_IVL':'sum'})
# churn_sub_quarterly_df = churn_sub_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
# churn_sub_quarterly_df = churn_sub_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF'
#                                          , 'PRE', 'PRE_T', 'PRE_D'
#                                          , 'POST_B2C', 'POST_B2C_VL', 'POST_B2C_IVL'
#                                          , 'POST_B2C_T', 'POST_B2C_VL_T', 'POST_B2C_IVL_T'
#                                          , 'POST_B2C_D', 'POST_B2C_VL_D', 'POST_B2C_IVL_D'
#                                          , 'FTTX', 'FTTX_VL', 'FTTX_IVL'
#                                          , 'TVS', 'TVS_VL', 'TVS_IVL']]
# churn_sub_quarterly_df_display = churn_sub_quarterly_df.copy()
# churn_sub_quarterly_df_display['ACTUAL_AS_OF'] = churn_sub_quarterly_df_display['ACTUAL_AS_OF'].astype(int)
# mod_col_list = churn_sub_quarterly_df_display.iloc[:, 3:].columns.tolist()
# for col in mod_col_list:
#     churn_sub_quarterly_df_display[col] = churn_sub_quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))
# print(f'\n{churn_sub_quarterly_df_display.to_string(max_cols=100)}') #max_rows=1000


''' Overview '''
churn_sub_quarterly_df = churn_sub_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'POST_B2C':'sum', 'POST_B2C_VL':'sum', 'POST_B2C_IVL':'sum', 'FTTX':'sum', 'FTTX_VL':'sum', 'FTTX_IVL':'sum', 'TVS':'sum', 'TVS_VL':'sum', 'TVS_IVL':'sum'})
churn_sub_quarterly_df = churn_sub_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
churn_sub_quarterly_df = churn_sub_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'POST_B2C', 'POST_B2C_VL', 'POST_B2C_IVL', 'FTTX', 'FTTX_VL', 'FTTX_IVL', 'TVS', 'TVS_VL', 'TVS_IVL']]
churn_sub_quarterly_df_display = churn_sub_quarterly_df.copy()
churn_sub_quarterly_df_display['ACTUAL_AS_OF'] = churn_sub_quarterly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = churn_sub_quarterly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    churn_sub_quarterly_df_display[col] = churn_sub_quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))
churn_sub_quarterly_df_display

,TM_KEY_QTR,PPN_TM,ACTUAL_AS_OF,PRE,POST_B2C,POST_B2C_VL,POST_B2C_IVL,FTTX,FTTX_VL,FTTX_IVL,TVS,TVS_VL,TVS_IVL
0,20241,2025-06-18 01:06:16,20240331,"9,491,554","871,395","318,353","576,878","118,742","43,223","72,945","48,913","37,318","13,347"
1,20242,2025-06-18 01:06:16,20240630,"8,344,376","718,051","262,796","454,251","114,261","43,677","70,807","40,970","35,580","5,046"
2,20243,2025-06-18 01:06:16,20240930,"8,899,074","664,232","248,339","433,905","118,872","44,269","67,909","48,795","5,040",355
3,20244,2025-06-18 01:06:16,20241231,"7,377,175","603,538","237,036","397,302","104,793","38,985","69,561","55,572","6,238",52
4,20251,2025-06-18 01:06:16,20250331,"7,728,433","553,494","225,501","341,127","104,815","38,083","68,600","47,799","5,854",35
5,20252,2025-06-18 01:06:16,20250616,"22,105,761","479,532","201,419","271,106","84,792","34,869","57,078","53,309","4,805",41


In [6]:
''' Churn Subs Monthly '''

# v_tm_key_yr = curr_yr
v_tm_key_yr = prev_yr

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'TB1S000200' #Prepaid Churn Subs : TMH
    , 'DB1S000200' #Prepaid Churn Subs : DTAC
    , 'B2S010200' #Postpaid Churn Subs B2C
    , 'TB2S010200' #Postpaid Churn Subs B2C : TMH
    , 'DB2S010200' #Postpaid Churn Subs B2C : DTAC
    , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    , 'TB2S010201' #Postpaid Churn Subs Voluntary B2C : TMH
    , 'DB2S010201' #Postpaid Churn Subs Voluntary B2C : DTAC
    , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TB2S010202' #Postpaid Churn Subs Involuntary B2C : TMH
    , 'DB2S010202' #Postpaid Churn Subs Involuntary B2C : DTAC
    # , 'TB2S010210' #Postpaid Churn Subs B2C : TMH (Most Used Location)
    # , 'TB2S010211' #Postpaid Churn Subs Voluntary B2C : TMH (Most Used Location)
    # , 'TB2S010212' #Postpaid Churn Subs Involuntary B2C : TMH (Most Used Location)
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    , 'TSER14102' #TVS Churn Subs Voluntary
    , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_sub_monthly_df = chk_src_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_YR']>=v_tm_key_yr].copy()
churn_sub_monthly_df = churn_sub_monthly_df.loc[churn_sub_monthly_df['METRIC_CD'].isin(v_metric_list)]

churn_sub_monthly_df['PRE'] = np.where(churn_sub_monthly_df['METRIC_CD']=='B1S000200', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['PRE_T'] = np.where(churn_sub_monthly_df['METRIC_CD']=='TB1S000200', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['PRE_D'] = np.where(churn_sub_monthly_df['METRIC_CD']=='DB1S000200', churn_sub_monthly_df['P'], 0)

churn_sub_monthly_df['POST_B2C'] = np.where(churn_sub_monthly_df['METRIC_CD']=='B2S010200', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['POST_B2C_VL'] = np.where(churn_sub_monthly_df['METRIC_CD']=='B2S010201', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['POST_B2C_IVL'] = np.where(churn_sub_monthly_df['METRIC_CD']=='B2S010202', churn_sub_monthly_df['P'], 0)

churn_sub_monthly_df['POST_B2C_T'] = np.where(churn_sub_monthly_df['METRIC_CD']=='TB2S010200', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['POST_B2C_VL_T'] = np.where(churn_sub_monthly_df['METRIC_CD']=='TB2S010201', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['POST_B2C_IVL_T'] = np.where(churn_sub_monthly_df['METRIC_CD']=='TB2S010202', churn_sub_monthly_df['P'], 0)

churn_sub_monthly_df['POST_B2C_D'] = np.where(churn_sub_monthly_df['METRIC_CD']=='DB2S010200', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['POST_B2C_VL_D'] = np.where(churn_sub_monthly_df['METRIC_CD']=='DB2S010201', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['POST_B2C_IVL_D'] = np.where(churn_sub_monthly_df['METRIC_CD']=='DB2S010202', churn_sub_monthly_df['P'], 0)

churn_sub_monthly_df['FTTX'] = np.where(churn_sub_monthly_df['METRIC_CD']=='TSER13100', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['FTTX_VL'] = np.where(churn_sub_monthly_df['METRIC_CD']=='TSER13102', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['FTTX_IVL'] = np.where(churn_sub_monthly_df['METRIC_CD']=='TSER13103', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['FTTX_IVL(CF)'] = np.where(churn_sub_monthly_df['METRIC_CD']=='TSER13104', churn_sub_monthly_df['P'], 0)

churn_sub_monthly_df['TVS'] = np.where(churn_sub_monthly_df['METRIC_CD']=='TSER14100', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['TVS_VL'] = np.where(churn_sub_monthly_df['METRIC_CD']=='TSER14102', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['TVS_IVL'] = np.where(churn_sub_monthly_df['METRIC_CD']=='TSER14103', churn_sub_monthly_df['P'], 0)


# ''' Full '''
# churn_sub_monthly_df = churn_sub_monthly_df.groupby('TM_KEY_MTH')\
#     .agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum'
#           , 'POST_B2C':'sum', 'POST_B2C_VL':'sum', 'POST_B2C_IVL':'sum'
#           , 'POST_B2C_T':'sum', 'POST_B2C_VL_T':'sum', 'POST_B2C_IVL_T':'sum'
#           , 'POST_B2C_D':'sum', 'POST_B2C_VL_D':'sum', 'POST_B2C_IVL_D':'sum'
#           , 'FTTX':'sum', 'FTTX_VL':'sum', 'FTTX_IVL':'sum'
#           , 'TVS':'sum', 'TVS_VL':'sum', 'TVS_IVL':'sum'})
# churn_sub_monthly_df = churn_sub_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
# churn_sub_monthly_df = churn_sub_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF'
#                                          , 'PRE', 'PRE_T', 'PRE_D'
#                                          , 'POST_B2C', 'POST_B2C_VL', 'POST_B2C_IVL'
#                                          , 'POST_B2C_T', 'POST_B2C_VL_T', 'POST_B2C_IVL_T'
#                                          , 'POST_B2C_D', 'POST_B2C_VL_D', 'POST_B2C_IVL_D'
#                                          , 'FTTX', 'FTTX_VL', 'FTTX_IVL'
#                                          , 'TVS', 'TVS_VL', 'TVS_IVL']]
# churn_sub_monthly_df_display = churn_sub_monthly_df.copy()
# churn_sub_monthly_df_display['ACTUAL_AS_OF'] = churn_sub_monthly_df_display['ACTUAL_AS_OF'].astype(int)
# mod_col_list = churn_sub_monthly_df_display.iloc[:, 3:].columns.tolist()
# for col in mod_col_list:
#     churn_sub_monthly_df_display[col] = churn_sub_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
# print(f'\n{churn_sub_monthly_df_display.to_string(max_cols=100)}') #max_rows=1000


''' Overview '''
churn_sub_monthly_df = churn_sub_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'POST_B2C':'sum', 'POST_B2C_VL':'sum', 'POST_B2C_IVL':'sum', 'FTTX':'sum', 'FTTX_VL':'sum', 'FTTX_IVL':'sum', 'TVS':'sum', 'TVS_VL':'sum', 'TVS_IVL':'sum'})
churn_sub_monthly_df = churn_sub_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
churn_sub_monthly_df = churn_sub_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'POST_B2C', 'POST_B2C_VL', 'POST_B2C_IVL', 'FTTX', 'FTTX_VL', 'FTTX_IVL', 'TVS', 'TVS_VL', 'TVS_IVL']]
churn_sub_monthly_df_display = churn_sub_monthly_df.copy()
churn_sub_monthly_df_display['ACTUAL_AS_OF'] = churn_sub_monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = churn_sub_monthly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    churn_sub_monthly_df_display[col] = churn_sub_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
churn_sub_monthly_df_display

,TM_KEY_MTH,PPN_TM,ACTUAL_AS_OF,PRE,POST_B2C,POST_B2C_VL,POST_B2C_IVL,FTTX,FTTX_VL,FTTX_IVL,TVS,TVS_VL,TVS_IVL
0,202401,2025-06-18 01:06:16,20240131,"3,191,108","335,195","118,348","231,754","40,468","16,454","24,014","14,004","10,196","3,820"
1,202402,2025-06-18 01:06:16,20240229,"3,076,385","281,024","102,361","180,543","38,531","14,057","24,474","12,774","7,689","6,795"
2,202403,2025-06-18 01:06:16,20240331,"3,224,061","255,176","97,644","164,581","39,743","12,712","24,457","22,135","19,433","2,732"
3,202404,2025-06-18 01:06:16,20240430,"3,045,247","247,654","90,553","155,090","38,455","13,522","24,520","20,819","18,602","2,217"
4,202405,2025-06-18 01:06:16,20240531,"2,757,874","250,447","92,059","151,641","38,397","14,955","23,442","16,541","14,731","1,810"
5,202406,2025-06-18 01:06:16,20240630,"2,541,255","219,950","80,184","147,520","37,409","15,200","22,845","3,610","2,247","1,019"
6,202407,2025-06-18 01:06:16,20240731,"3,214,945","232,711","91,260","145,540","40,306","15,934","22,944","9,003","1,136",22
7,202408,2025-06-18 01:06:16,20240831,"2,852,657","224,948","81,816","143,964","40,742","14,958","22,914","21,101","2,225",313
8,202409,2025-06-18 01:06:16,20240930,"2,831,472","206,573","75,263","144,401","37,824","13,377","22,051","18,691","1,679",20
9,202410,2025-06-18 01:06:16,20241031,"2,589,503","207,481","83,683","135,820","38,097","13,580","23,214","19,574","2,181",30


In [7]:
''' Churn Subs Weekly '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'TB1S000200' #Prepaid Churn Subs : TMH
    , 'DB1S000200' #Prepaid Churn Subs : DTAC
    , 'B2S010200' #Postpaid Churn Subs B2C
    , 'TB2S010200' #Postpaid Churn Subs B2C : TMH
    , 'DB2S010200' #Postpaid Churn Subs B2C : DTAC
    , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    , 'TB2S010201' #Postpaid Churn Subs Voluntary B2C : TMH
    , 'DB2S010201' #Postpaid Churn Subs Voluntary B2C : DTAC
    , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TB2S010202' #Postpaid Churn Subs Involuntary B2C : TMH
    , 'DB2S010202' #Postpaid Churn Subs Involuntary B2C : DTAC
    # , 'TB2S010210' #Postpaid Churn Subs B2C : TMH (Most Used Location)
    # , 'TB2S010211' #Postpaid Churn Subs Voluntary B2C : TMH (Most Used Location)
    # , 'TB2S010212' #Postpaid Churn Subs Involuntary B2C : TMH (Most Used Location)
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    , 'TSER14102' #TVS Churn Subs Voluntary
    , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_sub_weekly_df = chk_src_df[['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
churn_sub_weekly_df = churn_sub_weekly_df.loc[churn_sub_weekly_df['METRIC_CD'].isin(v_metric_list)]

churn_sub_weekly_df['PRE'] = np.where(churn_sub_weekly_df['METRIC_CD']=='B1S000200', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['PRE_T'] = np.where(churn_sub_weekly_df['METRIC_CD']=='TB1S000200', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['PRE_D'] = np.where(churn_sub_weekly_df['METRIC_CD']=='DB1S000200', churn_sub_weekly_df['P'], 0)

churn_sub_weekly_df['POST_B2C'] = np.where(churn_sub_weekly_df['METRIC_CD']=='B2S010200', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['POST_B2C_VL'] = np.where(churn_sub_weekly_df['METRIC_CD']=='B2S010201', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['POST_B2C_IVL'] = np.where(churn_sub_weekly_df['METRIC_CD']=='B2S010202', churn_sub_weekly_df['P'], 0)

churn_sub_weekly_df['POST_B2C_T'] = np.where(churn_sub_weekly_df['METRIC_CD']=='TB2S010200', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['POST_B2C_VL_T'] = np.where(churn_sub_weekly_df['METRIC_CD']=='TB2S010201', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['POST_B2C_IVL_T'] = np.where(churn_sub_weekly_df['METRIC_CD']=='TB2S010202', churn_sub_weekly_df['P'], 0)

churn_sub_weekly_df['POST_B2C_D'] = np.where(churn_sub_weekly_df['METRIC_CD']=='DB2S010200', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['POST_B2C_VL_D'] = np.where(churn_sub_weekly_df['METRIC_CD']=='DB2S010201', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['POST_B2C_IVL_D'] = np.where(churn_sub_weekly_df['METRIC_CD']=='DB2S010202', churn_sub_weekly_df['P'], 0)

churn_sub_weekly_df['FTTX'] = np.where(churn_sub_weekly_df['METRIC_CD']=='TSER13100', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['FTTX_VL'] = np.where(churn_sub_weekly_df['METRIC_CD']=='TSER13102', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['FTTX_IVL'] = np.where(churn_sub_weekly_df['METRIC_CD']=='TSER13103', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['FTTX_IVL(CF)'] = np.where(churn_sub_weekly_df['METRIC_CD']=='TSER13104', churn_sub_weekly_df['P'], 0)

churn_sub_weekly_df['TVS'] = np.where(churn_sub_weekly_df['METRIC_CD']=='TSER14100', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['TVS_VL'] = np.where(churn_sub_weekly_df['METRIC_CD']=='TSER14102', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['TVS_IVL'] = np.where(churn_sub_weekly_df['METRIC_CD']=='TSER14103', churn_sub_weekly_df['P'], 0)


# ''' Full '''
# churn_sub_weekly_df = churn_sub_weekly_df.groupby('TM_KEY_WK')\
#     .agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum'
#           , 'POST_B2C':'sum', 'POST_B2C_VL':'sum', 'POST_B2C_IVL':'sum'
#           , 'POST_B2C_T':'sum', 'POST_B2C_VL_T':'sum', 'POST_B2C_IVL_T':'sum'
#           , 'POST_B2C_D':'sum', 'POST_B2C_VL_D':'sum', 'POST_B2C_IVL_D':'sum'
#           , 'FTTX':'sum', 'FTTX_VL':'sum', 'FTTX_IVL':'sum'
#           , 'TVS':'sum', 'TVS_VL':'sum', 'TVS_IVL':'sum'})
# churn_sub_weekly_df = churn_sub_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
# churn_sub_weekly_df = churn_sub_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF'
#                                          , 'PRE', 'PRE_T', 'PRE_D'
#                                          , 'POST_B2C', 'POST_B2C_VL', 'POST_B2C_IVL'
#                                          , 'POST_B2C_T', 'POST_B2C_VL_T', 'POST_B2C_IVL_T'
#                                          , 'POST_B2C_D', 'POST_B2C_VL_D', 'POST_B2C_IVL_D'
#                                          , 'FTTX', 'FTTX_VL', 'FTTX_IVL'
#                                          , 'TVS', 'TVS_VL', 'TVS_IVL']]
# churn_sub_weekly_df_display = churn_sub_weekly_df.copy()
# churn_sub_weekly_df_display['ACTUAL_AS_OF'] = churn_sub_weekly_df_display['ACTUAL_AS_OF'].astype(int)
# mod_col_list = churn_sub_weekly_df_display.iloc[:, 3:].columns.tolist()
# for col in mod_col_list:
#     churn_sub_weekly_df_display[col] = churn_sub_weekly_df_display[col].apply(lambda x: format(x, ',.0f'))
# print(f'\n{churn_sub_weekly_df_display.to_string(max_cols=100)}') #max_rows=1000


''' Overview '''
churn_sub_weekly_df = churn_sub_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'POST_B2C':'sum', 'POST_B2C_VL':'sum', 'POST_B2C_IVL':'sum', 'FTTX':'sum', 'FTTX_VL':'sum', 'FTTX_IVL':'sum', 'TVS':'sum', 'TVS_VL':'sum', 'TVS_IVL':'sum'})
churn_sub_weekly_df = churn_sub_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
churn_sub_weekly_df = churn_sub_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'POST_B2C', 'POST_B2C_VL', 'POST_B2C_IVL', 'FTTX', 'FTTX_VL', 'FTTX_IVL', 'TVS', 'TVS_VL', 'TVS_IVL']]
churn_sub_weekly_df_display = churn_sub_weekly_df.copy()
churn_sub_weekly_df_display['ACTUAL_AS_OF'] = churn_sub_weekly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = churn_sub_weekly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    churn_sub_weekly_df_display[col] = churn_sub_weekly_df_display[col].apply(lambda x: format(x, ',.0f'))
churn_sub_weekly_df_display

,TM_KEY_WK,PPN_TM,ACTUAL_AS_OF,PRE,POST_B2C,POST_B2C_VL,POST_B2C_IVL,FTTX,FTTX_VL,FTTX_IVL,TVS,TVS_VL,TVS_IVL
0,2025018,2025-06-18 01:06:16,20250504,"302,458","27,170","10,643","16,178","7,727","2,289",2,"3,670",278,1
1,2025019,2025-06-18 01:06:16,20250511,"510,698","49,260","19,418","27,215","13,193","3,592",0,"4,746",460,8
2,2025020,2025-06-18 01:06:16,20250518,"519,290","50,095","17,970","25,429","8,076","3,232",2,"5,086",460,1
3,2025021,2025-06-18 01:06:16,20250525,"541,806","49,489","16,203","24,772","9,725","2,595","18,190","4,611",663,1
4,2025022,2025-06-18 01:06:16,20250601,"20,542,155","40,087","22,879","23,979","-1,047","3,495","5,404","5,354",408,3
5,2025023,2025-06-18 01:06:16,20250608,"4,683,931","49,654","23,579","27,180","14,695","3,956","1,905","6,653",630,1
6,2025024,2025-06-18 01:06:16,20250615,"-7,465,257","39,698","17,141","16,494","8,133","2,585","8,490","5,858",402,0
7,2025025,2025-06-18 01:06:16,20250616,"73,374",0,0,0,0,0,0,0,0,0


In [8]:
''' Churn Subs Daily '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'TB1S000200' #Prepaid Churn Subs : TMH
    , 'DB1S000200' #Prepaid Churn Subs : DTAC
    , 'B2S010200' #Postpaid Churn Subs B2C
    , 'TB2S010200' #Postpaid Churn Subs B2C : TMH
    , 'DB2S010200' #Postpaid Churn Subs B2C : DTAC
    , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    , 'TB2S010201' #Postpaid Churn Subs Voluntary B2C : TMH
    , 'DB2S010201' #Postpaid Churn Subs Voluntary B2C : DTAC
    , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TB2S010202' #Postpaid Churn Subs Involuntary B2C : TMH
    , 'DB2S010202' #Postpaid Churn Subs Involuntary B2C : DTAC
    # , 'TB2S010210' #Postpaid Churn Subs B2C : TMH (Most Used Location)
    # , 'TB2S010211' #Postpaid Churn Subs Voluntary B2C : TMH (Most Used Location)
    # , 'TB2S010212' #Postpaid Churn Subs Involuntary B2C : TMH (Most Used Location)
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    , 'TSER14102' #TVS Churn Subs Voluntary
    , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_sub_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
churn_sub_daily_df = churn_sub_daily_df.loc[churn_sub_daily_df['METRIC_CD'].isin(v_metric_list)]

churn_sub_daily_df['PRE'] = np.where(churn_sub_daily_df['METRIC_CD']=='B1S000200', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['PRE_T'] = np.where(churn_sub_daily_df['METRIC_CD']=='TB1S000200', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['PRE_D'] = np.where(churn_sub_daily_df['METRIC_CD']=='DB1S000200', churn_sub_daily_df['P'], 0)

churn_sub_daily_df['POST_B2C'] = np.where(churn_sub_daily_df['METRIC_CD']=='B2S010200', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['POST_B2C_VL'] = np.where(churn_sub_daily_df['METRIC_CD']=='B2S010201', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['POST_B2C_IVL'] = np.where(churn_sub_daily_df['METRIC_CD']=='B2S010202', churn_sub_daily_df['P'], 0)

churn_sub_daily_df['POST_B2C_T'] = np.where(churn_sub_daily_df['METRIC_CD']=='TB2S010200', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['POST_B2C_VL_T'] = np.where(churn_sub_daily_df['METRIC_CD']=='TB2S010201', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['POST_B2C_IVL_T'] = np.where(churn_sub_daily_df['METRIC_CD']=='TB2S010202', churn_sub_daily_df['P'], 0)

churn_sub_daily_df['POST_B2C_D'] = np.where(churn_sub_daily_df['METRIC_CD']=='DB2S010200', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['POST_B2C_VL_D'] = np.where(churn_sub_daily_df['METRIC_CD']=='DB2S010201', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['POST_B2C_IVL_D'] = np.where(churn_sub_daily_df['METRIC_CD']=='DB2S010202', churn_sub_daily_df['P'], 0)

churn_sub_daily_df['FTTX'] = np.where(churn_sub_daily_df['METRIC_CD']=='TSER13100', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['FTTX_VL'] = np.where(churn_sub_daily_df['METRIC_CD']=='TSER13102', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['FTTX_IVL'] = np.where(churn_sub_daily_df['METRIC_CD']=='TSER13103', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['FTTX_IVL(CF)'] = np.where(churn_sub_daily_df['METRIC_CD']=='TSER13104', churn_sub_daily_df['P'], 0)

churn_sub_daily_df['TVS'] = np.where(churn_sub_daily_df['METRIC_CD']=='TSER14100', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['TVS_VL'] = np.where(churn_sub_daily_df['METRIC_CD']=='TSER14102', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['TVS_IVL'] = np.where(churn_sub_daily_df['METRIC_CD']=='TSER14103', churn_sub_daily_df['P'], 0)


# ''' Full '''
# churn_sub_daily_df = churn_sub_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY'])\
#     .agg({'PPN_TM':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum'
#           , 'POST_B2C':'sum', 'POST_B2C_VL':'sum', 'POST_B2C_IVL':'sum'
#           , 'POST_B2C_T':'sum', 'POST_B2C_VL_T':'sum', 'POST_B2C_IVL_T':'sum'
#           , 'POST_B2C_D':'sum', 'POST_B2C_VL_D':'sum', 'POST_B2C_IVL_D':'sum'
#           , 'FTTX':'sum', 'FTTX_VL':'sum', 'FTTX_IVL':'sum'
#           , 'TVS':'sum', 'TVS_VL':'sum', 'TVS_IVL':'sum'})
# churn_sub_daily_df = churn_sub_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
# churn_sub_daily_df = churn_sub_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM'
#                                          , 'PRE', 'PRE_T', 'PRE_D'
#                                          , 'POST_B2C', 'POST_B2C_VL', 'POST_B2C_IVL'
#                                          , 'POST_B2C_T', 'POST_B2C_VL_T', 'POST_B2C_IVL_T'
#                                          , 'POST_B2C_D', 'POST_B2C_VL_D', 'POST_B2C_IVL_D'
#                                          , 'FTTX', 'FTTX_VL', 'FTTX_IVL'
#                                          , 'TVS', 'TVS_VL', 'TVS_IVL']]
# churn_sub_daily_df_display = churn_sub_daily_df.copy()
# mod_col_list = churn_sub_daily_df_display.iloc[:, 3:].columns.tolist()
# for col in mod_col_list:
#     churn_sub_daily_df_display[col] = churn_sub_daily_df_display[col].apply(lambda x: format(x, ',.0f'))
# print(f'\n{churn_sub_daily_df_display.to_string(max_cols=100)}') #max_rows=1000


''' Overview '''
churn_sub_daily_df = churn_sub_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE':'sum', 'POST_B2C':'sum', 'POST_B2C_VL':'sum', 'POST_B2C_IVL':'sum', 'FTTX':'sum', 'FTTX_VL':'sum', 'FTTX_IVL':'sum', 'TVS':'sum', 'TVS_VL':'sum', 'TVS_IVL':'sum'})
churn_sub_daily_df = churn_sub_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
churn_sub_daily_df = churn_sub_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE', 'POST_B2C', 'POST_B2C_VL', 'POST_B2C_IVL', 'FTTX', 'FTTX_VL', 'FTTX_IVL', 'TVS', 'TVS_VL', 'TVS_IVL']]
churn_sub_daily_df_display = churn_sub_daily_df.copy()
mod_col_list = churn_sub_daily_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    churn_sub_daily_df_display[col] = churn_sub_daily_df_display[col].apply(lambda x: format(x, ',.0f'))
churn_sub_daily_df_display

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,PRE,POST_B2C,POST_B2C_VL,POST_B2C_IVL,FTTX,FTTX_VL,FTTX_IVL,TVS,TVS_VL,TVS_IVL
0,202506,20250617,2025-06-18 01:06:16,0,0,0,0,0,0,0,0,0,0
1,202506,20250616,2025-06-18 01:06:16,"73,374",0,0,0,0,0,0,0,0,0
2,202506,20250615,2025-06-18 01:06:16,"69,818",0,0,0,0,0,0,967,43,0
3,202506,20250614,2025-06-18 01:06:16,"66,912","7,048","1,830","1,986",0,0,0,751,32,0
4,202506,20250613,2025-06-18 01:06:16,"62,866","4,491","2,421","2,680",-807,467,"4,210",806,63,0
5,202506,20250612,2025-06-18 01:06:16,"63,109","3,050","2,595","1,536","4,661",503,0,808,38,0
6,202506,20250611,2025-06-18 01:06:16,"-3,899,168","3,172","1,886","1,286",-575,544,0,814,52,0
7,202506,20250610,2025-06-18 01:06:16,"-3,908,637","16,100","5,567","4,850",-468,485,0,816,105,0
8,202506,20250609,2025-06-18 01:06:16,"79,843","5,837","2,842","4,156","5,322",586,"4,280",896,69,0
9,202506,20250608,2025-06-18 01:06:16,"4,095,399","3,130","2,273","3,961",-350,624,0,"1,168",181,0


## Products Summary

### Prep Monthly Data

In [9]:
''' Monthly Summary '''

monthly_df = chk_src_df.copy()
monthly_df = monthly_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
monthly_df = monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
monthly_df['DIFF (C-P)'] = monthly_df['C'] - monthly_df['P']

monthly_df_display = monthly_df.copy()
mod_col_list = monthly_df_display.iloc[:, 5:].columns.tolist()
for col in mod_col_list:
    monthly_df_display[col] = monthly_df_display[col].apply(lambda x: format(x, ',.0f'))

# monthly_df_display
# monthly_df_display.loc[monthly_df_display['TM_KEY_MTH']==202501]

In [10]:
''' Parameter '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth
# v_tm_key_mth = 202505

print(f'v_tm_key_mth: {v_tm_key_mth}')

v_tm_key_mth: 202505


### Prepaid

In [11]:
''' Prepaid '''

v_product_grp = 'Prepaid'

prepaid_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
prepaid_df = prepaid_df.loc[prepaid_df['TM_KEY_MTH']==v_tm_key_mth]
prepaid_df = prepaid_df.reset_index(drop=True)
prepaid_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202505,Prepaid,B1S000200,Prepaid Churn Subs,2025-06-17 23:53:41,0,"22,348,738","22,336,646","22,336,646",0,"-22,348,738"
1,202505,Prepaid,DB1S000200,Prepaid Churn Subs : DTAC,2025-06-17 23:53:41,0,"1,132,679","1,130,560","1,130,560",0,"-1,132,679"
2,202505,Prepaid,TB1S000200,Prepaid Churn Subs : TMH,2025-06-17 23:53:41,0,"21,216,059","21,206,086","21,206,086",0,"-21,216,059"


### Postpaid

In [12]:
''' Postpaid '''

v_product_grp = 'Postpaid'

postpaid_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_df = postpaid_df.loc[~postpaid_df['METRIC_NAME'].str.contains('B2C|B2B')]
postpaid_df = postpaid_df.loc[postpaid_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_df = postpaid_df.reset_index(drop=True)
postpaid_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)


In [13]:
''' Postpaid B2C '''

v_product_grp = 'Postpaid'

postpaid_b2c_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2c_df = postpaid_b2c_df.loc[postpaid_b2c_df['METRIC_NAME'].str.contains('B2C')]
postpaid_b2c_df = postpaid_b2c_df.loc[postpaid_b2c_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2c_df = postpaid_b2c_df.reset_index(drop=True)
postpaid_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202505,Postpaid,B2S010200,Postpaid Churn Subs B2C,2025-06-18 01:06:16,"211,802","211,802","211,477","211,477","211,477",0
1,202505,Postpaid,B2S010201,Postpaid Churn Subs Voluntary B2C,2025-06-18 01:06:16,"84,153","84,153","83,832","83,832","83,839",0
2,202505,Postpaid,B2S010202,Postpaid Churn Subs Involuntary B2C,2025-06-18 01:06:16,"115,298","115,298","115,294","115,294","115,294",0
3,202505,Postpaid,DB2S010200,Postpaid Churn Subs B2C : DTAC,2025-06-18 01:06:16,"76,091","76,091","75,999","75,999","75,999",0
4,202505,Postpaid,DB2S010201,Postpaid Churn Subs Voluntary B2C : DTAC,2025-06-18 01:06:16,"24,071","24,071","23,979","23,979","23,979",0
5,202505,Postpaid,DB2S010202,Postpaid Churn Subs Involuntary B2C : DTAC,2025-06-18 01:06:16,"39,669","39,669","39,669","39,669","39,669",0
6,202505,Postpaid,TB2S010200,Postpaid Churn Subs B2C : TMH,2025-06-18 01:06:16,"135,711","135,711","135,478","135,478","135,478",0
7,202505,Postpaid,TB2S010201,Postpaid Churn Subs Voluntary B2C : TMH,2025-06-18 01:06:16,"60,082","60,082","59,853","59,853","59,860",0
8,202505,Postpaid,TB2S010202,Postpaid Churn Subs Involuntary B2C : TMH,2025-06-18 01:06:16,"75,629","75,629","75,625","75,625","75,625",0
9,202505,Postpaid,TB2S010210,Postpaid Churn Subs B2C : TMH (Most Used Locat...,2025-06-18 01:06:16,0,0,0,0,0,0


In [14]:
''' Postpaid B2B '''

v_product_grp = 'Postpaid'

postpaid_b2b_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2b_df = postpaid_b2b_df.loc[postpaid_b2b_df['METRIC_NAME'].str.contains('B2B')]
postpaid_b2b_df = postpaid_b2b_df.loc[postpaid_b2b_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2b_df = postpaid_b2b_df.reset_index(drop=True)
postpaid_b2b_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202505,Postpaid,B2S020200,Postpaid Churn Subs B2B,2025-06-17 23:53:41,0,"24,158","24,157","24,157",0,"-24,158"
1,202505,Postpaid,B2S020201,Postpaid Churn Subs Voluntary B2B,2025-06-17 23:53:41,0,"23,031","23,031","23,031",0,"-23,031"
2,202505,Postpaid,B2S020202,Postpaid Churn Subs Involuntary B2B,2025-06-17 23:53:41,0,"1,127","1,126","1,126",0,"-1,127"
3,202505,Postpaid,DB2S020200,Postpaid Churn Subs B2B : DTAC,2025-06-17 23:53:41,0,"9,284","9,284","9,284",0,"-9,284"
4,202505,Postpaid,DB2S020201,Postpaid Churn Subs Voluntary B2B : DTAC,2025-06-17 23:53:41,0,"8,585","8,585","8,585",0,"-8,585"
5,202505,Postpaid,DB2S020202,Postpaid Churn Subs Involuntary B2B : DTAC,2025-06-17 23:53:41,0,699,699,699,0,-699
6,202505,Postpaid,TB2S020200,Postpaid Churn Subs B2B : TMH,2025-06-18 01:06:16,0,"14,874","14,872","14,872","14,872","-14,874"
7,202505,Postpaid,TB2S020201,Postpaid Churn Subs Voluntary B2B : TMH,2025-06-18 01:06:16,0,"14,446","14,445","14,445","14,445","-14,446"
8,202505,Postpaid,TB2S020202,Postpaid Churn Subs Involuntary B2B : TMH,2025-06-18 01:06:16,0,428,427,427,427,-428
9,202505,Postpaid,TB2S020210,Postpaid Churn Subs B2B : TMH (Most Used Locat...,2025-06-18 01:06:16,0,"14,874","14,873","14,873","14,873","-14,874"


### TOL

In [15]:
''' TOL '''

v_product_grp = 'TOL'

tol_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_df = tol_df.loc[tol_df['TM_KEY_MTH']==v_tm_key_mth]
tol_df = tol_df.reset_index(drop=True)
tol_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202505,TOL,TSER13100,FTTx Churn Subs,2025-06-18 01:06:16,0,"38,488","38,487","38,487","38,487","-38,488"
1,202505,TOL,TSER13102,FTTx Churn Subs Voluntary,2025-06-18 01:06:16,0,"14,562","14,561","14,561","14,561","-14,562"
2,202505,TOL,TSER13103,FTTx Churn Subs Involuntary,2025-06-18 01:06:16,0,"23,598","23,597","24,099","25,119","-23,598"
3,202505,TOL,TSER13104,FTTx Churn Subs Involuntary (Collection Forecast),2025-06-18 01:06:16,0,"57,851","57,850","57,850","57,850","-57,851"


### TVS

In [16]:
''' TVS '''

v_product_grp = 'TVS'

tvs_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tvs_df = tvs_df.loc[tvs_df['TM_KEY_MTH']==v_tm_key_mth]
tvs_df = tvs_df.reset_index(drop=True)
tvs_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202505,TVS,TSER14100,TVS Churn Subs,2025-06-17 23:53:41,0,"22,325","22,325","21,550",0,"-22,325"
1,202505,TVS,TSER14102,TVS Churn Subs Voluntary,2025-06-17 23:53:41,0,"2,123","22,311","21,580",0,"-2,123"
2,202505,TVS,TSER14103,TVS Churn Subs Involuntary,2025-06-17 23:53:41,0,14,14,6,0,-14


## ** Current Issue

In [17]:
''' Monthly : Prepaid Churn Subs : TMH (Align with BU) '''

v_metric_cd = 'TB1S000200'

issue_prepaid_tmh_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metric_cd]
issue_prepaid_tmh_df = issue_prepaid_tmh_df.loc[issue_prepaid_tmh_df['TM_KEY_MTH']>=202401]
issue_prepaid_tmh_df = issue_prepaid_tmh_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)

issue_prepaid_tmh_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202401,Prepaid,TB1S000200,Prepaid Churn Subs : TMH,2025-06-17 23:53:41,0,"1,411,781","1,217,672","1,201,070",0,"-1,411,781"
1,202402,Prepaid,TB1S000200,Prepaid Churn Subs : TMH,2025-06-17 23:53:41,0,"1,481,911","1,276,487","1,260,774",0,"-1,481,911"
2,202403,Prepaid,TB1S000200,Prepaid Churn Subs : TMH,2025-06-17 23:53:41,0,"1,445,434","1,292,312","1,276,558",0,"-1,445,434"
3,202404,Prepaid,TB1S000200,Prepaid Churn Subs : TMH,2025-06-17 23:53:41,0,"1,556,142","1,402,113","1,384,897",0,"-1,556,142"
4,202405,Prepaid,TB1S000200,Prepaid Churn Subs : TMH,2025-06-17 23:53:41,0,"1,420,875","-19,659,016","-19,674,697",0,"-1,420,875"
5,202406,Prepaid,TB1S000200,Prepaid Churn Subs : TMH,2025-06-17 23:53:41,0,"1,119,897","-20,783,458","-20,783,458",0,"-1,119,897"
6,202407,Prepaid,TB1S000200,Prepaid Churn Subs : TMH,2025-06-17 23:53:41,0,"1,606,946","-20,760,123","-20,760,123",0,"-1,606,946"
7,202408,Prepaid,TB1S000200,Prepaid Churn Subs : TMH,2025-06-17 23:53:41,0,"1,142,088","-20,671,886","-20,671,886",0,"-1,142,088"
8,202409,Prepaid,TB1S000200,Prepaid Churn Subs : TMH,2025-06-17 23:53:41,0,"1,376,492","-20,562,743","-20,562,743",0,"-1,376,492"
9,202410,Prepaid,TB1S000200,Prepaid Churn Subs : TMH,2025-06-17 23:53:41,0,"1,315,792","-20,429,168","-20,429,168",0,"-1,315,792"
